## Turkish Instructions Retrieval Embedding

In [1]:
# !pip install datasets

### Import Libraries

In [2]:
import sys
import os

import pandas as pd
import numpy as np
from src.utils import missing_values
import warnings
from datetime import datetime

from datasets import load_dataset

C:\Users\adem\miniconda3\envs\py_env_312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

warnings.filterwarnings('ignore')

#### Set Path and Constant Values

In [4]:
# Set Constants

# If use Google Colab
use_colab = False
if use_colab:
    # from google.colab import drive
    # drive.mount('/content/drive')    
    # ROOT_PATH = "/content/drive/MyDrive/Colab Notebooks/HW3"  # Google Drive için kök dizin
    # sys.path.append(os.path.join(ROOT_PATH, 'src'))
    device = "gpu"
else:
    ROOT_PATH = os.getcwd()
    device = "cpu"

CACHE_PATH = os.path.join(ROOT_PATH, 'cache')
DATA_PATH = os.path.join(ROOT_PATH, 'data')
RAW_DATA_PATH = os.path.join(DATA_PATH, 'raw')
EMBEDDING_DATA_PATH = os.path.join(DATA_PATH, 'embedding')

# Create Folders
os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(EMBEDDING_DATA_PATH, exist_ok=True)
os.makedirs(CACHE_PATH, exist_ok=True)

### Load Dataset: Turkish Instruction

In [5]:
ds = load_dataset("merve/turkish_instructions")

In [6]:
df_ins_data_all = ds['train'].to_pandas()
df_ins_data_all.head()

,Unnamed: 0,talimat,giriş,çıktı
0,0,Aşağıdaki bağlamda orijinal bir şey söyleyin: Bir iş görüşmesini yeni bitirdiniz ve işverene bu pozisyona olan ilginizi göstermek istiyorsunuz.,None,Ekibe katkıda bulunmaya başlamak ve bu alandaki becerilerimi daha da geliştirmek için çok istekliyim.
1,1,"Aşağıdaki makale için bir başlık önerin: Bu makale, işletmelerin müşterilerle iletişim kurma biçiminde devrim yaratmada internetin oynadığı rolü incelemektedir.",None,"""Dijital İletişimin Gücü: İnternet İş Ortamını Nasıl Dönüştürüyor"""
2,2,Şu konuyla ilgili 5 anahtar kelimeden oluşan bir liste oluşturun: iklim değişikliği,None,1. Karbon emisyonları 2. Yenilenebilir enerji 3. Küresel ısınma 4. İklim eylemi 5. Sürdürülebilir kalkınma
3,3,Bu cümleyi pasiften aktife değiştirin: Yeni politika belediye başkanı tarafından açıklandı.,None,Belediye başkanı yeni politikayı açıkladı.
4,4,Aşağıdaki fenomeni tanımlayan üç bilimsel terim tanımlayın: İki nesneyi birbirine çeken gizemli bir güç.,None,"Yerçekimi kuvveti, Newton'un evrensel çekim yasası ve elektrostatik kuvvet."


In [7]:
# 'Unnamed: 0' sütununu kaldır
if 'Unnamed: 0' in df_ins_data_all.columns:
    df_ins_data_all = df_ins_data_all.drop(columns=['Unnamed: 0'])

In [8]:
df_ins_data_all.columns

Index(['talimat', ' giriş', ' çıktı'], dtype='object')

In [9]:
# df_ins_data_all.columns[1] rename to "giris" and [2] rename to "cikis"
df_ins_data_all.rename(columns={df_ins_data_all.columns[0]: 'talimat'}, inplace=True)
df_ins_data_all.rename(columns={df_ins_data_all.columns[1]: 'giris'}, inplace=True)
df_ins_data_all.rename(columns={df_ins_data_all.columns[2]: 'cevap'}, inplace=True)

In [10]:
df_ins_data_all.to_csv(os.path.join(RAW_DATA_PATH, "turkish_instructions.csv"), index=False, encoding='utf-8')
df_ins_data_all.head()

,talimat,giris,cevap
0,Aşağıdaki bağlamda orijinal bir şey söyleyin: Bir iş görüşmesini yeni bitirdiniz ve işverene bu pozisyona olan ilginizi göstermek istiyorsunuz.,None,Ekibe katkıda bulunmaya başlamak ve bu alandaki becerilerimi daha da geliştirmek için çok istekliyim.
1,"Aşağıdaki makale için bir başlık önerin: Bu makale, işletmelerin müşterilerle iletişim kurma biçiminde devrim yaratmada internetin oynadığı rolü incelemektedir.",None,"""Dijital İletişimin Gücü: İnternet İş Ortamını Nasıl Dönüştürüyor"""
2,Şu konuyla ilgili 5 anahtar kelimeden oluşan bir liste oluşturun: iklim değişikliği,None,1. Karbon emisyonları 2. Yenilenebilir enerji 3. Küresel ısınma 4. İklim eylemi 5. Sürdürülebilir kalkınma
3,Bu cümleyi pasiften aktife değiştirin: Yeni politika belediye başkanı tarafından açıklandı.,None,Belediye başkanı yeni politikayı açıkladı.
4,Aşağıdaki fenomeni tanımlayan üç bilimsel terim tanımlayın: İki nesneyi birbirine çeken gizemli bir güç.,None,"Yerçekimi kuvveti, Newton'un evrensel çekim yasası ve elektrostatik kuvvet."


### Show Metadata, Shape, Statistics and Missing Table

In [11]:
df_ins_data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51563 entries, 0 to 51562
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   talimat  51563 non-null  object
 1   giris    19088 non-null  object
 2   cevap    51563 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [12]:
df_ins_data_all.shape

(51563, 3)

In [13]:
missing_values(df=df_ins_data_all, threshold=0, asc_sorting=False)

,Count,Percent
Columns,,
giris,32475,62.98121
talimat,0,0.00000
cevap,0,0.00000


In [14]:
col_list = df_ins_data_all.columns.to_list()

for col in col_list:
    df_ins_data_all[f"{col}Length"] = df_ins_data_all[col].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
    df_ins_data_all[f"{col}WordCount"] = df_ins_data_all[col].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

In [15]:
df_ins_data_all.head()

,talimat,giris,cevap,talimatLength,talimatWordCount,girisLength,girisWordCount,cevapLength,cevapWordCount
0,Aşağıdaki bağlamda orijinal bir şey söyleyin: Bir iş görüşmesini yeni bitirdiniz ve işverene bu pozisyona olan ilginizi göstermek istiyorsunuz.,None,Ekibe katkıda bulunmaya başlamak ve bu alandaki becerilerimi daha da geliştirmek için çok istekliyim.,144,19,0,0,102,14
1,"Aşağıdaki makale için bir başlık önerin: Bu makale, işletmelerin müşterilerle iletişim kurma biçiminde devrim yaratmada internetin oynadığı rolü incelemektedir.",None,"""Dijital İletişimin Gücü: İnternet İş Ortamını Nasıl Dönüştürüyor""",161,19,0,0,67,8
2,Şu konuyla ilgili 5 anahtar kelimeden oluşan bir liste oluşturun: iklim değişikliği,None,1. Karbon emisyonları 2. Yenilenebilir enerji 3. Küresel ısınma 4. İklim eylemi 5. Sürdürülebilir kalkınma,84,12,0,0,107,15
3,Bu cümleyi pasiften aktife değiştirin: Yeni politika belediye başkanı tarafından açıklandı.,None,Belediye başkanı yeni politikayı açıkladı.,92,11,0,0,43,5
4,Aşağıdaki fenomeni tanımlayan üç bilimsel terim tanımlayın: İki nesneyi birbirine çeken gizemli bir güç.,None,"Yerçekimi kuvveti, Newton'un evrensel çekim yasası ve elektrostatik kuvvet.",104,14,0,0,76,9


In [16]:
df_ins_data_all[["talimatLength", "girisLength", "cevapLength", "talimatWordCount", "girisWordCount", "cevapWordCount"]].describe(percentiles=[0.80, 0.85, 0.90, 0.95, 0.99]).T

,count,mean,std,min,50%,80%,85%,90%,95%,99%,max
talimatLength,51563.0,63.265403,26.986467,10.0,59.0,78.0,83.0,90.0,103.0,145.0,1056.0
girisLength,51563.0,23.468126,71.065212,0.0,0.0,34.0,45.0,62.0,103.0,289.0,2653.0
cevapLength,51563.0,279.215523,281.753389,1.0,196.0,486.0,548.7,628.0,775.0,1241.0,3607.0
talimatWordCount,51563.0,8.396563,3.836307,2.0,8.0,10.0,11.0,12.0,14.0,20.0,145.0
girisWordCount,51563.0,3.243392,9.317139,0.0,0.0,5.0,6.0,9.0,14.0,38.0,353.0
cevapWordCount,51563.0,37.181409,37.274826,1.0,27.0,64.0,72.0,82.0,102.0,166.0,495.0


### Split Data

In [17]:
df_ins_data_all['soru'] = df_ins_data_all['talimat'].fillna('') + ' ' + df_ins_data_all['giris'].fillna('')
df_ins_data_all[["talimat", "giris", "soru", "cevap"]].head()

,talimat,giris,soru,cevap
0,Aşağıdaki bağlamda orijinal bir şey söyleyin: Bir iş görüşmesini yeni bitirdiniz ve işverene bu pozisyona olan ilginizi göstermek istiyorsunuz.,None,Aşağıdaki bağlamda orijinal bir şey söyleyin: Bir iş görüşmesini yeni bitirdiniz ve işverene bu pozisyona olan ilginizi göstermek istiyorsunuz.,Ekibe katkıda bulunmaya başlamak ve bu alandaki becerilerimi daha da geliştirmek için çok istekliyim.
1,"Aşağıdaki makale için bir başlık önerin: Bu makale, işletmelerin müşterilerle iletişim kurma biçiminde devrim yaratmada internetin oynadığı rolü incelemektedir.",None,"Aşağıdaki makale için bir başlık önerin: Bu makale, işletmelerin müşterilerle iletişim kurma biçiminde devrim yaratmada internetin oynadığı rolü incelemektedir.","""Dijital İletişimin Gücü: İnternet İş Ortamını Nasıl Dönüştürüyor"""
2,Şu konuyla ilgili 5 anahtar kelimeden oluşan bir liste oluşturun: iklim değişikliği,None,Şu konuyla ilgili 5 anahtar kelimeden oluşan bir liste oluşturun: iklim değişikliği,1. Karbon emisyonları 2. Yenilenebilir enerji 3. Küresel ısınma 4. İklim eylemi 5. Sürdürülebilir kalkınma
3,Bu cümleyi pasiften aktife değiştirin: Yeni politika belediye başkanı tarafından açıklandı.,None,Bu cümleyi pasiften aktife değiştirin: Yeni politika belediye başkanı tarafından açıklandı.,Belediye başkanı yeni politikayı açıkladı.
4,Aşağıdaki fenomeni tanımlayan üç bilimsel terim tanımlayın: İki nesneyi birbirine çeken gizemli bir güç.,None,Aşağıdaki fenomeni tanımlayan üç bilimsel terim tanımlayın: İki nesneyi birbirine çeken gizemli bir güç.,"Yerçekimi kuvveti, Newton'un evrensel çekim yasası ve elektrostatik kuvvet."


In [18]:
df_sample_1 = df_ins_data_all[df_ins_data_all['giris'].notna()].sample(n=1000, random_state=42)
df_sample_1_idx = df_sample_1.index
df_sample_1.head()

df_sample_2 = df_ins_data_all[~(df_ins_data_all['giris'].notna())].sample(n=1000, random_state=42)
df_sample_2_idx = df_sample_2.index
df_sample_2.head()

df_sample = pd.concat([df_sample_1, df_sample_2])
df_sample['soru'] = df_sample['talimat'].fillna('') + " " + df_sample['giris'].fillna('')

df_ins_data_all[col].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

df_sample.to_csv(os.path.join(RAW_DATA_PATH, "sampled_turkish_instructions.csv"), index=False)

df_sample = df_sample[["talimat", "giris", "soru", "cevap"]].reset_index(drop=True)
df_sample = pd.read_csv(os.path.join(RAW_DATA_PATH, "sampled_turkish_instructions.csv"))
df_sample.head()

,talimat,giris,cevap,talimatLength,talimatWordCount,girisLength,girisWordCount,cevapLength,cevapWordCount,soru
0,Belirli bir konu hakkında bir makale yazın.,Konu: Zaman Yönetimi,"Zaman yönetimi, hayatta başarılı olmak için sahip olunması gereken önemli bir beceridir. Gününüzden en iyi şekilde yararlanabilmeniz için görevlere öncelik vermenizi ve zamanınızı en iyi şekilde değerlendirmenizi sağlar. Etkili zaman yönetimine sahip olmak aynı zamanda odaklanmanıza ve doğru yolda kalmanıza yardımcı olur ve ertelemeyi önler. Zamanınızı etkili bir şekilde yönetmenin bir yolu, bir program oluşturmak ve ona bağlı kalmaktır. Günün belirli saatlerini belirli etkinliklere ayırmanız ve bu saatlere saygı duymanız önemlidir. Örneğin spor yapmak istiyorsanız her gün aynı saatte bir saat ayırın ve o randevuya mutlaka uyun. Görevleriniz için ulaşılabilir hedefler ve zaman çizelgesi belirlediğinizden de emin olmalısınız. Bunu yaparak, daha büyük görevleri yönetilebilir parçalara ayırabilir ve odaklanmanızı ve motive olmanızı sağlayacak somut hedeflere ve teslim tarihlerine sahip olabilirsiniz. Zaman yönetimini iyileştirmenin bir başka yolu da düzenli kalmak ve bir ""yapılacaklar"" listesi oluşturmaktır. Yapılacaklar listesi, önemli görevleri takip etmenize ve en önemlilerine öncelik vermenize yardımcı olacaktır. Ayrıca, görevleri unutma olasılığını azaltır ve tamamlamanız gereken görevleri hatırlatır. Son olarak, dikkat dağıtıcı unsurları yönetmek, etkili zaman yönetimi için çok önemlidir. Verimli kalmak için, gecikmelere neden olabileceğinden ve üretkenliği engelleyebileceğinden, dikkat dağıtıcı unsurları sınırlamak önemlidir. Düzenli kalmak ve mola vermek bunu önlemeye yardımcı olabilir. Genel olarak, zaman yönetimi hayatta başarılı olmak için sahip olunması gereken çok önemli bir beceridir. Programlar oluşturarak, ulaşılabilir hedefler belirleyerek, düzenli kalarak ve dikkat dağıtıcı unsurları sınırlayarak, kişi zamanını etkili bir şekilde yönetebilir ve daha fazlasını başarabilir.",44,7,20,3,1817,228,Belirli bir konu hakkında bir makale yazın. Konu: Zaman Yönetimi
1,Aynı konudaki iki makaleyi alaka düzeyine ve doğruluğuna göre karşılaştırın.,"Madde 1: ""Büyük Bankalar İklim Değişikliğini Önemsiyor"" Madde 2: ""BP'nin İklim Değişikliğiyle Mücadele Planı""","Birinci madde, birçok büyük bankanın çabalarını tartışarak, bankacılık sektörünün iklim değişikliğine yaklaşımına daha geniş bir genel bakış sunuyor. Büyük bankaların motivasyonlarını ve girişimlerini ve potansiyel etkilerini derinlemesine inceler. İkinci madde, BP'nin belirli girişimlerine odaklanmaktadır ve şirketin iklim değişikliği planları hakkındaki bilgilerde daha ayrıntılı olabilir. Her iki makale de konuyla alakalı, ancak birinci makale bankacılık sektörünün iklim değişikliğine tepkisine daha kapsamlı bir bakış sunuyor.",77,10,110,14,534,63,"Aynı konudaki iki makaleyi alaka düzeyine ve doğruluğuna göre karşılaştırın. Madde 1: ""Büyük Bankalar İklim Değişikliğini Önemsiyor"" Madde 2: ""BP'nin İklim Değişikliğiyle Mücadele Planı"""
2,Sağlanan kelimeleri kullanarak çılgın bir kütüphane oluşturun.,"at, hindi, yelkenli",At macera yaşamak istediğinde bir hindiye atlar ve bir yelkenliyle yola çıkar.,63,7,20,3,79,12,"Sağlanan kelimeleri kullanarak çılgın bir kütüphane oluşturun. at, hindi, yelkenli"
3,"Bir senaryo verildiğinde, kullanıcının bir sonraki eyleminin ne olacağını tahmin edin",John yeni bir telefon aldı ve ekran kilidini ayarlıyor,"John muhtemelen bir ekran kilidi türü (örn. parola, PIN, parmak izi vb.) seçecek ve bunu ayarlamak için gereken bilgileri girecektir.",86,11,55,9,133,20,"Bir senaryo verildiğinde, kullanıcının bir sonraki eyleminin ne olacağını tahmin edin John yeni bir telefon aldı ve ekran kilidini ayarlıyor"
4,Aşağıdaki durumdaki kişi hakkında güzel bir şey söyleyin,Yanan bir binadan birini kurtaran bir itfaiyeci,"İtfaiyeci, başka birini kurtarmak için kendi güvenliğini riske attığı için kahraman ve özverilidir.",57,8,48,7,100,13,Aşağıdaki durumdaki kişi hakkında gü

In [19]:
df_sample.sample(5)

,talimat,giris,cevap,talimatLength,talimatWordCount,girisLength,girisWordCount,cevapLength,cevapWordCount,soru
1984,Resmi görünen bir e-posta konu satırı örneği oluşturun.,NaN,Eylem gerekli: A77541-A numaralı politikada acil durum değişikliği,56,8,0,0,67,8,Resmi görünen bir e-posta konu satırı örneği oluşturun.
1881,Doğal seçilim kavramını açıklayınız.,NaN,"Doğal seçilime bir örnek, türlerin zaman içinde çevrelerine uyum sağlama sürecidir. Örneğin, çok sayıda tohumun olduğu bir ortamdaki bir ispinoz popülasyonu, tohumları daha iyi kırmak için daha büyük gagalara sahip olacak şekilde zamanla gelişebilir. Zamanla, bu daha büyük gagalı ispinozlar, çevrelerinde hayatta kalma ve üreme şansları daha yüksek olduğundan, popülasyonun çoğunluğunu oluşturacaktır.",37,4,0,0,403,52,Doğal seçilim kavramını açıklayınız.
1323,"""Makine öğrenimi"" ifadesinin tanımını bulun",NaN,"Makine öğrenimi, sistemlerin performanslarını iyileştirmek için verileri ve kalıpları kullanma yeteneğine odaklanan bir yapay zeka dalıdır. Verileri analiz etmek için algoritmaları kullanmayı ve ardından sistemin nasıl karar veya tahminde bulunacağını öğrenmesini sağlayan tahmine dayalı modeller oluşturmak için elde edilen içgörüleri kullanmayı içerir. Açıkça programlanmadan bilgisayarların problem çözmede doğruluklarını nasıl artıracaklarını öğrenmelerini sağlayan bir yapay zeka biçimi olarak düşünülebilir.",44,5,0,0,513,58,"""Makine öğrenimi"" ifadesinin tanımını bulun"
5,İkinci dereceden denklemin çözümünü bulun.,x² - 7x + 10 = 0,İkinci dereceden denklemin çözümü x = 2 ve x = 5'tir.,43,5,16,7,54,11,İkinci dereceden denklemin çözümünü bulun. x² - 7x + 10 = 0
754,Verilen ifadeyi analiz edin ve görüşünüzü belirtin.,Yeni teknolojinin geliştirilmesi çevresel durumun iyileştirilmesine yardımcı olabilir.,"Yeni teknolojilerin geliştirilmesinin çevre üzerinde olumlu bir etkisi olabileceğine inanıyorum. Fosil yakıtlara olan bağımlılığın azaltılmasına yardımcı olabilecek ve sonuçta azaltılmış emisyonlara yol açabilecek yenilenebilir enerji kaynakları gibi enerji üretimi için alternatif çözümler sunabilir. Ayrıca kaynak tüketimini ve kirliliği azaltmak için verimliliği artırmaya yardımcı olabilir. Bununla birlikte, teknolojinin potansiyel olumsuz tarafları da vardır ve geliştirilmesinin çevre için istenmeyen veya istenmeyen sonuçlara yol açmamasını sağlamamız önemlidir.",52,7,87,8,571,64,Verilen ifadeyi analiz edin ve görüşünüzü belirtin. Yeni teknolojinin geliştirilmesi çevresel durumun iyileştirilmesine yardımcı olabilir.


In [20]:
df_sample.shape

(2000, 10)

### Embedding Models

#### Description

<!-- ![Alt text]("Embedding") -->

| **No** | **Model Adı**                           | **Parametre Sayısı** | **Boyut Sayısı (Dimension Size)** | **Açıklama (Description)**                                                                                     | **Özel Amaçlı Eğitim**                   | **Eğitim Veri Kümesi**                                                        | **Desteklenen Diller**        |
|--------|----------------------------------------|----------------------|----------------------------------|---------------------------------------------------------------------------------------------------------------|-----------------------------------------|-----------------------------------------------------------------------------|---------------------------------|
| 1      | **sentence-transformers/all-MiniLM-L12-v2** | 33.4M                | 384                              | Cümleleri ve kısa paragrafları 384 boyutlu yoğun bir vektör uzayına dönüştüren bir modeldir. Bu model, cümle benzerliği, kümeleme ve anlamsal arama gibi görevlerde kullanılabilir. ([HuggingFace](https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2)) | Microsoft/MiniLM-L12-H384-uncased modeli üzerinde ince ayar yapılmıştır. | Cümle çiftleri (NLI ve Paraphrase Corpus)                                     | İngilizce                       |
| 2      | **jinaai/jina-embeddings-v3**           | 572M                 | 1536                             | Jina AI tarafından geliştirilen bu model, metinleri 1536 boyutlu vektörlere dönüştürmek için tasarlanmıştır. ([HuggingFace](https://huggingface.co/jinaai/jina-embeddings-v3))         | -                                       | Özel büyük ölçekli çok dilli veri kümesi                                      | Çok Dilli (50+ dil)            |
| 3      | **intfloat/multilingual-e5-large-instruct** | 560M                 | 1024                             | Çok dilli metinleri 1024 boyutlu vektörlere dönüştürebilen bir modeldir. ([HuggingFace](https://huggingface.co/intfloat/multilingual-e5-large-instruct))                                | -                                       | mC4, CCNet, MassiveText gibi büyük veri kümeleri                              | Çok Dilli (100+ dil)           |
| 4      | **BAAI/bge-m3**                         | -                    | 1024                             | Beijing Academy of Artificial Intelligence (BAAI) tarafından geliştirilen bu model, metinleri 1024 boyutlu vektörlere dönüştürmek için tasarlanmıştır. ([HuggingFace](https://huggingface.co/BAAI/bge-m3)) | -                                       | OpenAI GPT benzeri veri kümesi (web crawl)                                    | İngilizce, Çince               |
| 5      | **ytu-ce-cosmos/turkish-colbert**       | 111M                 | 768                              | Yıldız Teknik Üniversitesi tarafından geliştirilen bu model, Türkçe metinleri 128 boyutlu vektörlere dönüştürmek için tasarlanmıştır. ([HuggingFace](https://huggingface.co/ytu-ce-cosmos/turkish-colbert)) | Türkçe metinler üzerinde özel olarak eğitilmiştir. | Türkçe metin veri kümeleri                                                  | Türkçe                         |
| 6      | **thenlper/gte-large**                  | 335M                 | 1024                              | The NLP ER tarafından geliştirilen bu model, metinleri 768 boyutlu vektörlere dönüştürmek için tasarlanmıştır. ([HuggingFace](https://huggingface.co/thenlper/gte-large))             | -                                       | Çoklu cümle çiftleri ve NLI veri kümeleri                                     | İngilizce                       |


In [22]:
from sentence_transformers import SentenceTransformer

embedding_models = [
    'sentence-transformers/all-MiniLM-L12-v2',
    "jinaai/jina-embeddings-v3",
    "intfloat/multilingual-e5-large-instruct",
    "BAAI/bge-m3",
    "ytu-ce-cosmos/turkish-colbert",
    "thenlper/gte-large"
]

In [21]:
def load_or_download_model(model_name, device):
    model_path = os.path.join(CACHE_PATH, model_name)
    if os.path.exists(model_path):
        print(f"Loading Cached Model: {model_name}")
        return SentenceTransformer(model_path, device=device, trust_remote_code=True)
    else:
        print(f"Downloading Model: {model_name}")
        model = SentenceTransformer(model_name, device=device, trust_remote_code=True)
        model.save(model_path)
        return model

In [23]:
for idx, model_name in enumerate(embedding_models):
    try:
        embedding_start_time = datetime.now()
        print(f"{idx+1}-Embedding Model Name: {model_name} - {embedding_start_time}")
        model = load_or_download_model(model_name, device)
        
        # Sorular için embedding
        print("Sorular için embedding hesaplanıyor...")
        # if embedding_models == "ytu-ce-cosmos/turkish-colbert":
        #     df_sample['soru'] = df_sample['soru'].str.lower()        
        question_embeddings = model.encode(df_sample['soru'].tolist(), show_progress_bar=True)
        question_embedding_dim = len(question_embeddings[0])
        print(f"Embedding Dimension: {question_embedding_dim}")
        question_columns = [f'question_embedding_{i}' for i in range(question_embedding_dim)]
        question_df = pd.DataFrame(question_embeddings, columns=question_columns)
        question_df = pd.concat([df_sample[['soru']], question_df], axis=1)
        question_output_file = os.path.join(
          EMBEDDING_DATA_PATH, f"question_{model_name.replace('/', '__')}_embeddings.csv.gz"
        )
        question_df.to_csv(question_output_file, compression='gzip', index=False)
        
        # Cevaplar için embedding
        print("Cevaplar için embedding hesaplanıyor...")
        # if embedding_models == "ytu-ce-cosmos/turkish-colbert":
        #     df_sample['soru'] = df_sample['soru'].str.lower()        
        answer_embeddings = model.encode(df_sample['cevap'].tolist(), show_progress_bar=True)
        answer_embedding_dim = len(answer_embeddings[0])
        print(f"Embedding Dimension: {answer_embedding_dim}")
        answer_columns = [f'answer_embedding_{i}' for i in range(answer_embedding_dim)]
        answer_df = pd.DataFrame(answer_embeddings, columns=answer_columns)
        answer_df = pd.concat([df_sample[['cevap']], answer_df], axis=1)
        answer_output_file = os.path.join(
          EMBEDDING_DATA_PATH, f"answer_{model_name.replace('/', '__')}_embeddings.csv.gz"
        )
        answer_df.to_csv(answer_output_file, compression='gzip', index=False)
        
        print(f"{model_name} için embedding işlemi tamamlandı. Embedding Duration: {datetime.now() - embedding_start_time}")
        print("+ " * 15)
    except Exception as exc:
        print(f"{model_name} için hata oluştu: {exc}")
        print("x " * 15)

1-Embedding Model Name: sentence-transformers/all-MiniLM-L12-v2 - 2024-12-15 22:29:11.579497
Loading cached model: sentence-transformers/all-MiniLM-L12-v2
Sorular için embedding hesaplanıyor...


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:46<00:00,  1.34it/s]


Embedding Dimension: 384
Cevaplar için embedding hesaplanıyor...


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [01:18<00:00,  1.25s/it]


Embedding Dimension: 384
sentence-transformers/all-MiniLM-L12-v2 için embedding işlemi tamamlandı. Embedding Duration: 0:02:19.621986
+ + + + + + + + + + + + + + + 
2-Embedding Model Name: jinaai/jina-embeddings-v3 - 2024-12-15 22:31:31.202844
Loading cached model: jinaai/jina-embeddings-v3
Sorular için embedding hesaplanıyor...


...

Embedding Dimension: 768
ytu-ce-cosmos/turkish-colbert için embedding işlemi tamamlandı. Embedding Duration: 0:09:07.072922
+ + + + + + + + + + + + + + + 
6-Embedding Model Name: thenlper/gte-large - 2024-12-16 00:13:09.091106
Loading cached model: thenlper/gte-large
Sorular için embedding hesaplanıyor...


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [08:45<00:00,  8.34s/it]


Embedding Dimension: 1024
Cevaplar için embedding hesaplanıyor...


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [20:45<00:00, 19.77s/it]


Embedding Dimension: 1024
thenlper/gte-large için embedding işlemi tamamlandı. Embedding Duration: 0:29:54.291171
+ + + + + + + + + + + + + + + 


#### Embedding Vectors

In [24]:
# df_emb_data_que_1 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_sentence-transformers__all-MiniLM-L12-v2_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_1 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_sentence-transformers__all-MiniLM-L12-v2_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_2 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_jinaai__jina-embeddings-v3_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_2 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_jinaai__jina-embeddings-v3_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_3 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_intfloat__multilingual-e5-large-instruct_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_3 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_intfloat__multilingual-e5-large-instruct_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_4 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_BAAI__bge-m3_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_4 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_BAAI__bge-m3_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_5 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_ytu-ce-cosmos__turkish-colbert_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_5 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_ytu-ce-cosmos__turkish-colbert_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_6 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_thenlper__gte-large_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_6 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_thenlper__gte-large_embeddings.csv.gz"), compression='gzip')

In [25]:
for idx, f in enumerate(os.listdir(EMBEDDING_DATA_PATH)):
    # print(f)
    # if f.split("_")[0] == "answer":
    if f.split("_")[0] in ["answer", "question"]:
        df_emb_data = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, f), compression='gzip')
        print(f"{idx+1} - Emb. File Name:{f} & Shape: {df_emb_data.shape}")
        print(df_emb_data.head(0))
        # print(f"Columns:{df_emb_data.columns.to_list()}")
        print("+ " * 25)

2 - Emb. File Name:answer_BAAI__bge-m3_embeddings.csv.gz & Shape: (2000, 1025)
Empty DataFrame
Columns: [cevap, answer_embedding_0, answer_embedding_1, answer_embedding_2, answer_embedding_3, answer_embedding_4, answer_embedding_5, answer_embedding_6, answer_embedding_7, answer_embedding_8, answer_embedding_9, answer_embedding_10, answer_embedding_11, answer_embedding_12, answer_embedding_13, answer_embedding_14, answer_embedding_15, answer_embedding_16, answer_embedding_17, answer_embedding_18, answer_embedding_19, answer_embedding_20, answer_embedding_21, answer_embedding_22, answer_embedding_23, answer_embedding_24, answer_embedding_25, answer_embedding_26, answer_embedding_27, answer_embedding_28, answer_embedding_29, answer_embedding_30, answer_embedding_31, answer_embedding_32, answer_embedding_33, answer_embedding_34, answer_embedding_35, answer_embedding_36, answer_embedding_37, answer_embedding_38, answer_embedding_39, answer_embedding_40, answer_embedding_41, answer_embedding